<a href="https://colab.research.google.com/github/nstaudac/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module1-afirstlookatdata/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [0]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things

test_list = [10, 20, 30, 40, 50, 60]

# this is a test message # 

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [0]:
# TODO - your code here
# Use what we did live in lecture as an example

# importing necessary libraries

import pandas as pd
import numpy as np




In [2]:
# uploaded raw csv to google colab

from google.colab import files
uploaded = files.upload()

Saving single-color-names.csv to single-color-names.csv


In [8]:
# load data-set to google colab

df = pd.read_csv("single-color-names.csv")
df.head(5)


,Color Name
0,Air Force Blue (Raf)
1,Air Force Blue (Usaf)
2,Air Superiority Blue
3,Alabama Crimson
4,Alice Blue


In [11]:
# determining shape of data, features and observations

df.shape

(865, 1)

In [22]:
# determining data type
df.dtypes


Color Name    object
dtype: object

In [23]:
# checking last five rows of data

df.tail(5)

,Color Name
860,Yellow Orange
861,Yellow (Process)
862,Yellow (Ryb)
863,Zaffre
864,Zinnwaldite Brown


In [30]:
# determining total number of Null values

df.isnull().sum().sum()

0

Answers to Assignment Questions

1. First, in order to work any of the data from the dataset, I had to tell the computer to bring in two program libraries to get the necessary tools to start analyzing the data. Second, after downloading the dataset I had to send the file to the colab notebook via the upload function. Third, I opened the file via the read_csv function and checked the first five rows with df.head(). Afterwards, I checked how many columns and rows it had with df.shape, reviwed the data types in the data set, examined the last five rows with df.tail(), and lastly made sure there were no missing data values and added all missing values together with isnull function combined with the .sum( ) function. 


2. The most challenging part was trying to determine the best way to access the datasets themselves. Some of the datasets did not have url links or had incomplete csv files listed in the main page. 


3. The most interesting thing I learned was the chaining of .sum() together with the isnull function to determine the total amount of missing values. 


4. I would like to explore datasets further via creating histograms or other various plots within the matplotlib library, as the concepts behind them are more familar to me from prior data science experimentation. 

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

2.  What was the most challenging part of what you did?

3.  What was the most interesting thing you learned?

4.  What area would you like to explore with more time?




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.